In [3]:
import cv2
import os
from segment import FrameOrg,Frame
import matplotlib.pyplot as plt
import time
dir_path = "img\\neg"
picture_path = os.listdir(dir_path)
score = []
st = time.time()
for pic_file in picture_path:
    img = cv2.imread(os.path.join(dir_path,pic_file))
    frame = FrameOrg(img)
    # scorei = (frame.color_score+frame.symmetry_score)/2.0
    # score.append(scorei)
print(f"15张图片的计算时间（轻量化前）：{time.time()-st}s")

st = time.time()
for pic_file in picture_path:
    img = cv2.imread(os.path.join(dir_path,pic_file))
    frame = Frame(img)
    # print(pic_file)
    # print(frame.color_score)
    # print(frame.symmetry_score)
    # print('\n')
    # scorei = (frame.color_score+frame.symmetry_score)/2.0
    # score.append(scorei)
print(f"15张图片的计算时间（轻量化后）：{time.time()-st}s")
# score = []
# #创建图并命名
# plt.figure('Scatter fig')
# ax = plt.gca()
# #设置x轴、y轴名称
# ax.set_xlabel('x')
# ax.set_ylabel('y')
# x = list(range(15))

# #画散点图，以x_list中的值为横坐标，以y_list中的值为纵坐标
# #参数c指定点的颜色，s指定点的大小,alpha指定点的透明度

# plt.show()


# for i,s in enumerate(score):
#     if i/5 == 0:
#         p

# ax.scatter(x_list, y_list, c='r', s=20, alpha=0.5)

RuntimeError: The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 0